In [ ]:
import os
import warnings
from parmed.exceptions import GromacsWarning
warnings.simplefilter("ignore", GromacsWarning)
warnings.simplefilter("ignore", UserWarning)

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import mbuild as mb
import metamds as mds
import mdtraj as md

import nglview
from ipywidgets.widgets import Box

%matplotlib inline
!rm -rf output/ *.top *.gro

In [ ]:
def build_monolayer(chain_length, n_molecules, **kwargs):
    from mbuild.examples import AlkaneMonolayer
    pattern = mb.Random2DPattern(n_molecules)
    monolayer = AlkaneMonolayer(pattern, tile_x=1, tile_y=1, 
                                chain_length=chain_length)
    mb.translate(monolayer, [0, 0, 2])
    return monolayer

In [ ]:
def create_run_script(build_func, forcefield, input_dir, **kwargs):
    compound = build_func(**kwargs)
    name = compound.name
    em = os.path.join(input_dir, 'em.mdp')
    nvt = os.path.join(input_dir, 'nvt.mdp')
    gro = '{name}.gro'.format(name=name)
    top = '{name}.top'.format(name=name)

    box = compound.boundingbox
    compound.periodicity += np.array([0, 0, 5 * box.lengths[2]])
    compound.save(top, forcefield=forcefield, overwrite=True)

    em_grompp = 'gmx grompp -f {mdp} -c {gro} -p {top} -o em.tpr'.format(mdp=em, gro=gro, top=top)
    em_mdrun = 'gmx mdrun -v -deffnm em -ntmpi 1'

    nvt_grompp = 'gmx grompp -f {mdp} -c em.gro -p {top} -o nvt.tpr'.format(mdp=nvt, top=top)
    nvt_mdrun = 'gmx mdrun -v -deffnm nvt -ntmpi 1'

    script = (em_grompp, em_mdrun, nvt_grompp, nvt_mdrun)
    return script

In [ ]:
# Initialize simulation with a template and some metadata
sim = mds.Simulation(name='monolayer', 
                     template=create_run_script,
                     input_dir='static_input_files',
                     output_dir='output')

chain_lengths = [8, 12, 16, 20]
for length in chain_lengths:
    parameters = {'chain_length': length,
                  'n_molecules': 100, 
                  'forcefield': 'OPLS-aa',
                  'build_func': build_monolayer}
    # Parameterize our simulation template
    sim.parametrize(**parameters)

In [ ]:
# Run
REMOTE_EXECUTION=True
if REMOTE_EXECUTION:
    sim.execute_all(hostname='rahman.vuse.vanderbilt.edu', username='imodels')
else:
    sim.execute_all()

In [ ]:
if REMOTE_EXECUTION:
    sim.sync_all()

In [ ]:
task_1 = next(sim.tasks())
trj_path = os.path.join(task_1.output_dir, 'nvt.xtc')
top_path = os.path.join(task_1.output_dir, 'em.gro')
traj = md.load(trj_path, top=top_path)
print(traj)

In [ ]:
t = nglview.MDTrajTrajectory(traj)
w = nglview.NGLWidget(t)
w.representations = [{'type':'ball+stick'}]
Box(children=(w,))

In [ ]:
average_S2 = list()
for task in sim.tasks():
    # Load up the trajectory
    trj_path = os.path.join(task.output_dir, 'nvt.xtc')
    top_path = os.path.join(task.output_dir, 'em.gro')
    traj = md.load(trj_path, top=top_path)
    
    # Nematic order parameter
    atoms_per_chain = int((traj.n_atoms - 1800) / 100)
    chain_indices = [[n+x for x in range(atoms_per_chain)] 
                     for n in range(1800, traj.n_atoms, atoms_per_chain)]
    s2 = md.compute_nematic_order(traj, indices=chain_indices)
    average_S2.append(np.mean(s2))
    
    
plt.plot(chain_lengths, average_S2)
plt.xlabel('chain length (# carbon atoms)')
plt.ylabel('Mean S2')